In [33]:
import pandas as pd
import numpy as np
from scipy import optimize

import os

def iones(nombre):
    nombre = str(nombre)
    path = os.getcwd() + '/templates/'
    dict_csv = { os.path.basename(x).split('.')[0]: path + '/' + x for x in os.listdir(path) if x.endswith('.csv') }
    ion_df = pd.read_csv(dict_csv[nombre]).fillna(0)

    indice = ion_df['GreenHow']

    copy = ion_df.copy()
    del copy['GreenHow']
    del copy['Ingredientes']

    A = copy.to_numpy()
    AT = A.transpose()
    
    return ion_df, indice, A, AT


def objetivo(nombre):
    path = os.getcwd() + '/inputs/'
    dict_csv = { os.path.basename(x).split('.')[0]: path + '/' + x for x in os.listdir(path) if x.endswith('.csv') }
    df = pd.read_csv(dict_csv[nombre]).fillna(0)

    target = df.iloc[0].to_numpy()[2:]
    const = df['VALOR'].to_numpy()[1:]

    return target, const

def corrida(indice, constraints, matriz):

    lista = []
    index = []

    for i in range(len(indice)):
        if constraints[i] == 1:
            print('Añadiendo: ' + indice[i])
            lista.append(matriz[i])
            index.append(indice[i])
        if constraints[i] == 0:
            print('No se utilizara : ' + indice[i])
    A = np.array(lista)

    AT = A.transpose()

    return A, AT, index


def loss(x0, k, A):
    lista = [sum((k - np.dot(x0, A))**2)/k.shape[0] for i in range(x0.shape[0])]
    
    return sum(lista)


def estefania(guardar=False):

    variedad = input('Ingrese el nombre de la variedad: ')

    df, indice, matriz, trans = iones()

    elementos = list(df.iloc[0][2:].index)

    target, const = objetivo(variedad)

    A, AT, nuevoindice = corrida(indice, const, matriz)

    x = np.random.rand(A.shape[0]) 

    print('X = ' , x)

    loss(x, target, A)

    res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
                        method='L-BFGS-B', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='nelder-mead', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='SLSQP', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='BFGS', bounds=[(0,None)]*x.shape[0])    


    # resultados
    d = pd.DataFrame({'fertilizante': nuevoindice, 'g/l': res.x})

    # % de error

    error = (target - np.dot(res.x, A))*100/1

    err = [round(num, 2) for num in error]

    e = pd.DataFrame({'error[%]': err})

    resultados = pd.DataFrame(elementos, columns=['elemento'])
    resultados = resultados.join(e)

    resultados = d.join(resultados)

    if guardar == True:

        current = os.getcwd()
        # guardar resultados
        resultados.to_csv(current + '/outputs/' + 'resultados' + variedad + '.csv', index=False)


    return resultados


</b>
Electronic<br>
Salt <br>
Target <br>
Estimator <br>
For <br>
Advanced <br>
Nutrition <br>
In <br>
Agriculture <br>

# ESTEFANIA

In [ ]:
estefania(guardar=True)

### paso a paso: 

## Cargamos los datos de las sales

In [34]:
df, indice, matriz, trans = iones('iones')

print('Las sales disponibles para el estudio son: ')
print(indice)

elementos = list(df.iloc[0][2:].index)


Las sales disponibles para el estudio son: 
0                           Ácido nítrico
1                         Ácido fosfórico
2                         Ácido sulfúrico
3              Nitrato de amonio, Nitamks
4         Nitrato de Potasio más Mg, KERF
5     Nitrato de potasio más S, Nitroksul
6               Nitrato de calcio samurai
7                    Nitrato de calcio bi
8                     Nitrato de magnesio
9       Fosfato monoamónico, Map colossal
10     Fosfato monopotásico, Mkp colossal
11                      Sulfato de amonio
12           Sulfato de potasio, Solucros
13                      Sulfato de calcio
14            Sulfato de magnesio, Sulmag
15                      Sulfato de fierro
16                   Sulfato de manganeso
17                        Sulfato de zinc
18                       Sulfato de cobre
19                         Quelato Ca 95%
20                Quelato Fe 6%, microhow
21            Quelato de Mn 13%, microhow
22            Quelato de Zn 15%,

## Cargamos la solución objetivo

In [35]:
variedad = 'general'

target, const = objetivo(variedad)


for i in range(len(elementos)):
    print('El objetivo del ' + elementos[i] + ' es: ' +  str(target[i]) + ' g/l')

El objetivo del H+ es: 0.0 g/l
El objetivo del NO3- es: 9.38693625193657 g/l
El objetivo del NH4+ es: 4.17800052831859 g/l
El objetivo del H2PO4- es: 1.29141561816095 g/l
El objetivo del K+ es: 5.62803786134561 g/l
El objetivo del Ca+2 es: 3.74270173162333 g/l
El objetivo del Mg+2 es: 2.0571898786258 g/l
El objetivo del SO4-2 es: 2.1830656479027 g/l
El objetivo del Fe es: 0.03581341212284 g/l
El objetivo del Mn es: 0.018201674554059 g/l
El objetivo del Zn es: 0.0022932623950832 g/l
El objetivo del Cu es: 0.0015736631731344 g/l
El objetivo del B es: 0.0462534690101758 g/l
El objetivo del Mo es: 0.0005211590577444 g/l
El objetivo del Co es: 0.0 g/l
El objetivo del Cl- es: 0.0 g/l
El objetivo del Na+ es: 0.0 g/l


## Hacemos la corrida con los fertilizantes seleccionados

In [36]:
A, AT, nuevoindice = corrida(indice, const, matriz)

No se utilizara : Ácido nítrico
No se utilizara : Ácido fosfórico
No se utilizara : Ácido sulfúrico
Añadiendo: Nitrato de amonio, Nitamks
No se utilizara : Nitrato de Potasio más Mg, KERF
Añadiendo: Nitrato de potasio más S, Nitroksul
No se utilizara : Nitrato de calcio samurai
Añadiendo: Nitrato de calcio bi
Añadiendo: Nitrato de magnesio
No se utilizara : Fosfato monoamónico, Map colossal
Añadiendo: Fosfato monopotásico, Mkp colossal
No se utilizara : Sulfato de amonio
No se utilizara : Sulfato de potasio, Solucros
No se utilizara : Sulfato de calcio
No se utilizara : Sulfato de magnesio, Sulmag
No se utilizara : Sulfato de fierro
No se utilizara : Sulfato de manganeso
No se utilizara : Sulfato de zinc
No se utilizara : Sulfato de cobre
No se utilizara : Quelato Ca 95%
Añadiendo: Quelato Fe 6%, microhow
Añadiendo: Quelato de Mn 13%, microhow
Añadiendo: Quelato de Zn 15%, microhow
Añadiendo: Quelato Cu 15%, microhow
No se utilizara : Quelato full mix
No se utilizara : Quelato full mix

In [37]:
for i in range(len(nuevoindice)):
    print('El ' + nuevoindice[i] + ' es el # ' + str(i)  )


El Nitrato de amonio, Nitamks es el # 0
El Nitrato de potasio más S, Nitroksul es el # 1
El Nitrato de calcio bi es el # 2
El Nitrato de magnesio es el # 3
El Fosfato monopotásico, Mkp colossal es el # 4
El Quelato Fe 6%, microhow es el # 5
El Quelato de Mn 13%, microhow es el # 6
El Quelato de Zn 15%, microhow es el # 7
El Quelato Cu 15%, microhow es el # 8
El Ácido bórico, abo es el # 9
El Molibdato de sodio, MOST es el # 10


## Creamos el vector a optimizar 

In [45]:
x = np.random.rand(A.shape[0])

print('Vector aleatoreo de longitud ' + str(len(x)) + ':', x)

Vector aleatoreo de longitud 11: [0.55362696 0.68793985 0.45822177 0.78440135 0.32227414 0.98063712
 0.28008804 0.20527871 0.39624263 0.48982631 0.82887098]


# Función perdida 

In [46]:
print('A es de dimension: ', A.shape)
print('AT es de dimension: ', AT.shape)
print('target es de dimension: ', target.shape)
print('x es de dimension: ', x.shape)


A es de dimension:  (11, 17)
AT es de dimension:  (17, 11)
target es de dimension:  (17,)
x es de dimension:  (11,)


In [47]:
loss(x, target, A)

190.3338914831569

In [48]:
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='SLSQP')
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='BFGS')
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='Nelder-Mead')
res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True}, 
                        method='L-BFGS-B', bounds=[(0,1)]*x.shape[0])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.90334D+02    |proj g|=  9.80637D-01

At iterate    1    f=  1.04773D+02    |proj g|=  1.00000D+00

At iterate    2    f=  1.30686D+01    |proj g|=  8.37287D-01

At iterate    3    f=  1.08001D+01    |proj g|=  8.13367D-01

At iterate    4    f=  8.82730D+00    |proj g|=  9.65742D-01

At iterate    5    f=  8.45666D+00    |proj g|=  9.65742D-01

At iterate    6    f=  7.78739D+00    |proj g|=  6.98056D-01

At iterate    7    f=  7.76844D+00    |proj g|=  7.22875D-01

At iterate    8    f=  7.75614D+00    |proj g|=  7.00824D-01

At iterate    9    f=  7.75099D+00    |proj g|=  4.42798D-01

At iterate   10    f=  7.74660D+00    |proj g|=  2.08020D-01

At iterate   11    f=  7.74413D+00    |proj g|=  1.98342D-01

At iterate   12    f=  7.73867D+00    |proj g|=  1.79842D-01

At iterate   13    f=  7.7

In [49]:
# resultados
d = pd.DataFrame({'fertilizante': nuevoindice, 'g/l': res.x})

# % de error

error = (target - np.dot(res.x, A))*100/1

err = [round(num, 2) for num in error]

e = pd.DataFrame({'error[%]': err})

resultados = pd.DataFrame(elementos, columns=['elemento'])
resultados = resultados.join(e)

resultados = d.join(resultados)
resultados

,fertilizante,g/l,elemento,error[%]
0,"Nitrato de amonio, Nitamks",0.296237,H+,0.00
1,"Nitrato de potasio más S, Nitroksul",0.300915,NO3-,-85.34
2,Nitrato de calcio bi,0.496127,NH4+,61.30
3,Nitrato de magnesio,0.053602,H2PO4-,-69.26
4,"Fosfato monopotásico, Mkp colossal",0.273518,K+,70.28
5,"Quelato Fe 6%, microhow",0.033232,Ca+2,172.82
6,"Quelato de Mn 13%, microhow",0.007668,Mg+2,180.08
7,"Quelato de Zn 15%, microhow",0.001096,SO4-2,190.91
8,"Quelato Cu 15%, microhow",0.000593,Fe,0.01
9,"Ácido bórico, abo",0.002866,Mn,0.01


## Gradiente descendente

In [16]:
def calculate_step_size(gradient):
    """
    Function to calculate the step size (learning rate) for gradient-based optimization methods.

    Parameters:
    - gradient (ndarray): The gradient vector.

    Returns:
    - step_size (float): The step size.
    """
    step_size = 0.1  # Fixed step size value

    return step_size


def calculate_gradient(loss_function, x):
    """
    Function to calculate the gradient of a given loss function using the finite difference method.

    Parameters:
    - loss_function (function): The loss function for which the gradient is to be calculated.
    - x (ndarray): The point at which to calculate the gradient.

    Returns:
    - gradient (ndarray): The gradient vector.
    """
    epsilon = 1e-6  # Small value for numerical differentiation

    gradient = np.zeros_like(x)
    
    for i in range(len(x)):
        perturbation = np.zeros_like(x)
        perturbation[i] = epsilon
        
        loss_plus = loss_function(x + perturbation)
        loss_minus = loss_function(x - perturbation)
        
        gradient[i] = (loss_plus - loss_minus) / (2 * epsilon)
    
    return gradient


def find_minimum_loss(loss_function, initial_guess, iterations):
    """
    Function to search for the minimum of a given loss function.

    Parameters:
    - loss_function (function): The loss function to minimize.
    - initial_guess (ndarray): The initial value to start the search from.
    - iterations (int): The number of iterations to perform.

    Returns:
    - minimum_value (float): The minimum value found.
    """
    current_value = initial_guess

    for _ in range(iterations):
        gradient = calculate_gradient(loss_function, current_value)
        step_size = calculate_step_size(gradient)  # You can use a fixed step size or apply more advanced optimization techniques like line search or adaptive step sizes.
        current_value -= step_size * gradient

    minimum_value = current_value
    
    return minimum_value



In [ ]:
minimum_solution = find_minimum_loss(loss, initial_guess, iterations)

print(minimum_solution)

## Cargamos los datos de las sales

In [17]:
df, indice, matriz, trans = iones('iones_2')

print('Las sales disponibles para el estudio son: ')
print(indice)

elementos = list(df.iloc[0][2:].index)

Las sales disponibles para el estudio son: 
0                    CALCIO BI SC 25 KG
1             calcio samurai con amonio
2                 MAP COLOSSAL SC 25 KG
3                 MKP COLOSSAL SC 25 KG
4     KERF ( Nitrato de Potasio mas Mg)
5                      NITAMKS SC 25 KG
6                   nitrato de magnesio
7                NITROKSUL NKS SC 25 KG
8                     SOLUCROS SC 25 KG
9                     sulfato de amonio
10                      SULMAG SC 50 KG
11                 ***Sulfato de calcio
12                     ***Quelato Ca95%
Name: GreenHow, dtype: object


## Cargamos la solución objetivo

In [18]:
variedad = 'kale-001'

target, const = objetivo(variedad)


for i in range(len(elementos)):
    print('El objetivo del ' + elementos[i] + ' es: ' +  str(target[i]) + ' g/l')

El objetivo del NO3- es: 0.0 g/l
El objetivo del SO4-2 es: 9.38693625193657 g/l
El objetivo del H2PO4- es: 4.17800052831859 g/l
El objetivo del K+ es: 1.12998866589083 g/l
El objetivo del Ca+2 es: 5.37221795855717 g/l
El objetivo del Mg+2 es: 3.74270173162333 g/l
El objetivo del NH4+ es: 1.85147089076322 g/l


## Hacemos la corrida con los fertilizantes seleccionados

In [11]:
A, AT, nuevoindice = corrida(indice, const, matriz)

No se utilizara : CALCIO BI SC 25 KG
No se utilizara : calcio samurai con amonio
No se utilizara : MAP COLOSSAL SC 25 KG
Añadiendo: MKP COLOSSAL SC 25 KG
Añadiendo: KERF ( Nitrato de Potasio mas Mg)
Añadiendo: NITAMKS SC 25 KG
Añadiendo: nitrato de magnesio
Añadiendo: NITROKSUL NKS SC 25 KG
Añadiendo: SOLUCROS SC 25 KG
Añadiendo: sulfato de amonio
Añadiendo: SULMAG SC 50 KG
Añadiendo: ***Sulfato de calcio
Añadiendo: ***Quelato Ca95%


In [12]:
for i in range(len(nuevoindice)):
    print('El ' + nuevoindice[i] + ' es el # ' + str(i)  )


El MKP COLOSSAL SC 25 KG es el # 0
El KERF ( Nitrato de Potasio mas Mg) es el # 1
El NITAMKS SC 25 KG es el # 2
El nitrato de magnesio es el # 3
El NITROKSUL NKS SC 25 KG es el # 4
El SOLUCROS SC 25 KG es el # 5
El sulfato de amonio es el # 6
El SULMAG SC 50 KG es el # 7
El ***Sulfato de calcio es el # 8
El ***Quelato Ca95% es el # 9


## Creamos el vector a optimizar 

In [13]:
x = np.random.rand(A.shape[0])

print('Vector aleatoreo de longitud ' + str(len(x)) + ':', x)

Vector aleatoreo de longitud 10: [0.1359944  0.53705857 0.73646049 0.3332377  0.28121106 0.61788253
 0.8981857  0.66509429 0.29406626 0.41393602]


# Función perdida 

In [14]:
print('A es de dimension: ', A.shape)
print('AT es de dimension: ', AT.shape)
print('target es de dimension: ', target.shape)
print('x es de dimension: ', x.shape)


A es de dimension:  (10, 7)
AT es de dimension:  (7, 10)
target es de dimension:  (17,)
x es de dimension:  (10,)


In [15]:
loss(x, target, A)

ValueError: operands could not be broadcast together with shapes (17,) (7,) 